<a href="https://colab.research.google.com/github/cnhzgb/MachineL/blob/main/VGG11_32*32_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipdb
!pip install einops
import ipdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from einops import rearrange
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 426.5 kB/s eta 0:00:00


In [2]:
trans = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) # transforms.RandomHorizontalFlip(), transforms.RandomGrayscale()
dataset = datasets.CIFAR10(root="dataset/", transform=trans, download=True, train=True) # 5W张图片, 10种分类
loader = DataLoader(dataset, batch_size=100, shuffle=True)
batch_num,(image, label) = next(enumerate(loader))
print(len(dataset.classes), len(dataset), image.shape, label.shape) # 10; 5W; 100,3,32,32; 100

trans_test = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
dataset_test = datasets.CIFAR10(root="dataset/", transform=trans_test, download=True, train=False) # 1W张图片
loader_test = DataLoader(dataset_test, batch_size=100, shuffle=True)
criterion_test = nn.CrossEntropyLoss()

100%|██████████| 170498071/170498071 [00:02<00:00, 60691074.10it/s]


Extracting dataset/cifar-10-python.tar.gz to dataset/
10 50000 torch.Size([100, 3, 224, 224]) torch.Size([100])
Files already downloaded and verified


In [3]:
# https://zhuanlan.zhihu.com/p/87555358
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.conv11 = nn.Conv2d(3,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv21 = nn.Conv2d(64,128,3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv31 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv32 = nn.Conv2d(256, 256, 3,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv41 = nn.Conv2d(256, 512, 3,padding=1)
        self.conv42 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv51 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv52 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(512*7*7,1024)
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024,10)


    def forward(self,x):
        x = F.relu(self.conv11(x)) # => ,64,224,224
        x = self.pool1(x) # => ,64,112,112

        x = F.relu(self.conv21(x)) # => ,128,112,112
        x = self.pool2(x) # => ,128,56,56

        x = F.relu(self.conv31(x)) # => ,256,56,56
        x = F.relu(self.conv32(x))
        x = self.pool3(x) # => ,256,28,28

        x = F.relu(self.conv41(x)) # => ,512,28,28
        x = F.relu(self.conv42(x))
        x = self.pool4(x) # => ,512,14,14

        x = F.relu(self.conv51(x)) # => ,512,14,14
        x = F.relu(self.conv52(x))
        x = self.pool5(x) # => ,512,7,7

        x = x.view(-1,512*7*7)
        x = F.relu(self.fc1(x)) # => ,4096
        x = F.relu(self.fc2(x)) # => ,4096
        x = self.fc3(x) # => ,10
        x = F.softmax(x, dim=1)

        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
         MaxPool2d-2         [-1, 64, 112, 112]               0
            Conv2d-3        [-1, 128, 112, 112]          73,856
         MaxPool2d-4          [-1, 128, 56, 56]               0
            Conv2d-5          [-1, 256, 56, 56]         295,168
            Conv2d-6          [-1, 256, 56, 56]         590,080
         MaxPool2d-7          [-1, 256, 28, 28]               0
            Conv2d-8          [-1, 512, 28, 28]       1,180,160
            Conv2d-9          [-1, 512, 28, 28]       2,359,808
        MaxPool2d-10          [-1, 512, 14, 14]               0
           Conv2d-11          [-1, 512, 14, 14]       2,359,808
           Conv2d-12          [-1, 512, 14, 14]       2,359,808
        MaxPool2d-13            [-1, 512, 7, 7]               0
           Linear-14                 [-

In [ ]:
for epoch in range(1,10):
  total_loss = []
  errorTotal = 0
  for batch_idx, (img, label) in enumerate(loader):
    img = img.to(device)
    label = label.to(device)
    outputs = model(img)
    loss = criterion(outputs, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss.append(loss.item())
    maxV,maxIdx = outputs.max(dim=1)
    errorNum = torch.sum(torch.ne(maxIdx, label)).item()
    errorTotal += errorNum

    if(batch_idx % 100 == 99):
      print("epoch:{} batch:{} loss:{:.2f} mean:{:.2f} error:{}/100 errorTotal:{}/{} {:.2f}%".format(
          epoch, batch_idx, loss, np.mean(total_loss), errorNum, errorTotal, (batch_idx+1)*100, errorTotal/(batch_idx+1)))